In [3]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from pytexit import py2tex
from IPython.display import Markdown
from gekko import GEKKO
import pandas as pd
from matplotlib import rcParams
import os
import sys
from copy import copy


%matplotlib widget
#%matplotlib inline

# Look for custom python codes in the Radigan Engineering Vault...
sys.path.insert(1, 'C:\\\\RadiganVault\\\\Python')

import RadiganEngineeringDefaults as RE
from RadiganEngineering_Materials_Library import *
from RadiganEngineeringTorsionSpring import *
from RadiganEngineeringBellevilleSpring import *

# Use higher quality figures in PDF exports
#rcParams['figure.dpi']= 300
#rcParams['figure.figsize'] = [8, 6]
set_matplotlib_formats('png', 'pdf')

In [8]:
Module4 = GEKKO(remote=False)    # create GEKKO model for dx/dt
#Module4 = GEKKO()    # create GEKKO model for dx/dt


t_sim = 10 * ureg.sec # Number of seconds to simulate
points_per_second = 10

Module4.time = np.linspace(0, t_sim.magnitude, (t_sim.magnitude * points_per_second) + 1) * ureg.sec # time points

n_points = len(Module4.time)
#print(Module4.time)
# dt is in the units of ________.


# Inputs (4):
# qf  = Inlet Volumetric Flowrate (L/min)
# q   = Outlet Volumetric Flowrate (L/min)
# Caf = Feed Concentration (mol/L)
# Tf  = Feed Temperature (K)

# Inlet Volumetric Flowrate (L/min)
qf_vals = np.ones(n_points)* 5.2
qf_vals[30:] = 5.5
qf_vals[50:] = 5.1
qf = Module4.Param(qf_vals)

# Outlet Volumetric Flowrate (L/min)
q_vals = np.ones(n_points)*5.0
q = Module4.Param(q_vals)

# Feed Concentration (mol/L)
Caf_vals = np.ones(n_points)*1.0
Caf_vals[30:] = 0.5
Caf = Module4.Param(Caf_vals)

# Feed Temperature (K)
Tf_vals = np.ones(n_points)*300.0
Tf_vals[30:] = 315.0
Tf_vals[70:] = 325.0
Tf  = Module4.Param(Tf_vals)


# States (3): (Variables, with initial conditions)
Volume  = Module4.Var(1.0) # Volume (L)
Ca = Module4.Var(0.0) # Concentration of A (mol/L)
T  = Module4.Var(350.0) # Temperature (K)

# Parameters:
# Reaction
rA = Module4.Param(0.0)

# Equations (3):
# --> Mass balance: volume derivative
Module4.Equation(Volume.dt() == qf - q )
#dVdt = qf - q

# --> Species balance: concentration derivative
# Chain rule: d(V*Ca)/dt = Ca * dV/dt + V * dCa/dt
#dCadt = (qf*Caf - q*Ca)/Volume - rA - (Ca*dVdt/Volume)
Module4.Equation(Ca.dt() == (qf*Caf - q*Ca)/Volume - rA - (Ca*Volume.dt()/Volume) )

# --> Energy balance: temperature derivative
# Chain rule: d(V*T)/dt = T * dV/dt + V * dT/dt
#dTdt = (qf*Tf - q*T)/Volume - (T*dVdt/Volume)
Module4.Equation(T.dt() == (qf*Tf - q*T)/Volume - (T*Volume.dt()/Volume) )



# solve ODEs and plot
# https://gekko.readthedocs.io/en/latest/imode.html
Module4.options.IMODE = 4 # dynamic simulation
#Module4.options.IMODE = 3 # Steady State simulation

# Type: Integer, Input/Output
# Default Value: 0
# Description: Cold start model: 0=warm start, 1=cold start, 2=decompose problem
#Module4.options.COLDSTART = 2

#Module4.options.SOLVER = 1
#Module4.options.MAX_ITER = 1000

#---------------------
#Module4.options.NODES = 6
#---------------------

#Module4.options.TIME_SHIFT=0

Module4.solve(disp=False)

#print(Module4.time)

#print('Starting Level = {0:.3f}, Final Level = {1:.3f}'.format(Level.value[0], Level.value[-1]))




ii = 40
print('Concentration (Ca) at Time = {0} is {1:.3f}'.format(ii,Ca[ii]))

# Plot the inputs and results
plt.figure()

plt.subplot(3,2,1)
plt.plot(Module4.time,qf,'b--',linewidth=3)
plt.plot(Module4.time,q,'b:',linewidth=3)
plt.ylabel('Flow Rates (L/min)')
plt.legend(['Inlet','Outlet'],loc='best')
plt.grid(True)

plt.subplot(3,2,3)
plt.plot(Module4.time,Caf,'r--',linewidth=3)
plt.ylabel('Caf (mol/L)')
plt.legend(['Feed Concentration'],loc='best')
plt.grid(True)

plt.subplot(3,2,5)
plt.plot(Module4.time,Tf,'k--',linewidth=3)
plt.ylabel('Tf (K)')
plt.legend(['Feed Temperature'],loc='best')
plt.xlabel('Time (min)')
plt.grid(True)

plt.subplot(3,2,2)
plt.plot(Module4.time,Volume,'b-',linewidth=3)
plt.ylabel('Volume (L)')
plt.legend(['Volume'],loc='best')
plt.grid(True)

plt.subplot(3,2,4)
plt.plot(Module4.time,Ca,'r-',linewidth=3)
plt.ylabel('Ca (mol/L)')
plt.legend(['Concentration'],loc='best')
plt.grid(True)

plt.subplot(3,2,6)
plt.plot(Module4.time,T,'k-',linewidth=3)
plt.ylabel('T (K)')
plt.legend(['Temperature'],loc='best')
plt.xlabel('Time (min)')
plt.grid(True)

plt.show()
plt.suptitle('Tank Mixing Simulation with Temperature and Concentration')


Concentration (Ca) at Time = 40 is 0.529


C:\ProgramData\Anaconda3\lib\site-packages\gekko\gk_write_files.py:144: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  csv_data = np.hstack(('time',np.array(self.time).flatten().astype(object)))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0.98, 'Tank Mixing Simulation with Temperature and Concentration')